In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 보스턴 주택 가격 데이터 세트를 Peceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
* 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [ ]:
!pip install scikit-learn==1.0.2

In [5]:
import sklearn

print(sklearn.__version__)

1.0.2


In [6]:
from sklearn.datasets import load_boston

boston = load_boston()
print(load_boston())

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 1

/opt/conda/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

In [7]:
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Weight와 Bias의 Update 값을 계산하는 함수 생성.
* w1은 RM(방의 계수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* bias는 Bias
* N은 입력 데이터 건수
![](https://raw.githubusercontent.com/chulminkw/CNN_PG/main/utils/images/Weight_update.png)


In [8]:
# gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수. 
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨. 
# 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 loss로 반환.
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    
    #Number of data
    N = len(target)
    #Predicted Value
    predicted = bias + w1 * rm + w2 * lstat
    #Difference between actual and predicted values
    diff = target - predicted

    #w1, w2의 업데이트값(-n * (dLoss(w)/dw1))
    #.T는 세로 행렬을 가로 행렬로 변환하기 위해 수행한다. 행렬 곱 ㅡ * ㅣ 을 하기 위해서.
    w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))

    #Set to obtain bias based on array.
    #bias도 행렬로 만들어 주기 위해서 ones 함수를 사용한다.
    #Np.ones((3,2)) -> [[1. 1.], [1. 1.], [1. 1.]]
    #Np.ones((N,)) -> [1, 1, 1, ...N개... , 1]
    bias_factors = np.ones((N,))
    
    #bias의 업데이트값(-n * (dLoss(bias)/dbias))
    #.T는 세로 행렬을 가로 행렬로 변환하기 위해 수행한다. 행렬 곱 ㅡ * ㅣ 을 하기 위해서.
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))

    #회귀오류측정을 위해 모든 오차의 제곱을 해서 그 평균을 산출한다.
    mse_loss = np.mean(np.square(diff))
    
    return bias_update, w1_update, w2_update, mse_loss

### Gradient Descent 를 적용하는 함수 생성
* iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용. 

In [9]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    w1 = np.zeros((1,)) # np연산의 기본 데이터타입은 float이므로 [0.]이 출력된다.
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    #features는 Numpy 배열이고 2차원 배열입니다.
    #features[0,1]은 1행 2열의 값입니다.
    #features[:,0]에서 :는 모든 행을 뜻합니다. 즉, 1열의 모든 행 값을 의미합니다.
    #features = np.array(
    #[[1,2]
    #[3,4]
    #[5,6]])
    #features[0,1] = 2
    #features[:,0] = [1,3,5]
    #위에 코드에서는 1열의 도믄 행 값을 rm이라 칭하고 2열의 모든 행 값을 lstat라 칭한다는 것을 의미한다.
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # weight/bias update 값 계산 
        bias_update, w1_update, w2_update, mse_loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        # weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

### Gradient Descent 적용
* 신경망은 데이터를 정규화/표준화 작업을 미리 선행해 주어야 함. 
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])
print(scaled_features[:15])

[[0.57750527 0.08967991]
 [0.5479977  0.2044702 ]
 [0.6943859  0.06346578]
 [0.65855528 0.03338852]
 [0.68710481 0.09933775]
 [0.54972217 0.09602649]
 [0.4696302  0.29525386]
 [0.50028741 0.48068433]
 [0.39662771 0.7781457 ]
 [0.46809734 0.424117  ]
 [0.53956697 0.51655629]
 [0.46905537 0.31843267]
 [0.44606246 0.38576159]
 [0.45755892 0.18018764]
 [0.48572523 0.23537528]]


In [11]:
w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000
w1: [0.252369] w2: [0.10914761] bias: [0.45065613] loss: 592.1469169960474
Epoch: 2 / 1000
w1: [0.4982605] w2: [0.21458377] bias: [0.8890071] loss: 564.6567515182813
Epoch: 3 / 1000
w1: [0.73785103] w2: [0.31641055] bias: [1.315389] loss: 538.6424811965484
Epoch: 4 / 1000
w1: [0.97131229] w2: [0.41472723] bias: [1.73012873] loss: 514.0245946883915
Epoch: 5 / 1000
w1: [1.1988113] w2: [0.50963037] bias: [2.13354428] loss: 490.7278647125017
Epoch: 6 / 1000
w1: [1.42051052] w2: [0.60121392] bias: [2.52594493] loss: 468.6811172230454
Epoch: 7 / 1000
w1: [1.63656797] w2: [0.68956922] bias: [2.90763152] loss: 447.81701302090454
Epoch: 8 / 1000
w1: [1.84713735] w2: [0.77478516] bias: [3.27889669] loss: 428.07184113172934
Epoch: 9 / 1000
w1: [2.05236818] w2: [0.85694818] bias: [3.64002506] loss: 409.3853233168043
Epoch: 10 / 1000
w1: [2.25240586] w2: [0.93614234] bias: [3.9912935] loss: 391.700429116892
Epoch: 11 / 1000
w1: [2.44739187] w2: [1.012

### 계산된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산. 

In [12]:
#scaled_features[:, 0] = rm
#scaled_features[:, 1] = lstat
##### 위의 연산으로 구해진 최종 w1, w2, bias #######
#[w1 = 18.88518124] [w2 = -12.82053747] [bias = 16.54239941]
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.298946
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.270017
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.842337
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.551276
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.244935
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.692891
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.626138
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.827786
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.056539
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.945095
